In [168]:
import numpy as np
import os
import pandas as pd
import sklearn.datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.base import BaseEstimator
from tqdm.notebook import trange, tqdm
import math

In [151]:
def ohe(target, classes):
    return (classes == target[:, None]).astype(int)

In [152]:
class Elcl:
    def __init__(self):
        pass
        
    def fit(self, X_train, y_train, num_classes, max_elems):
        self.classes = num_classes
        self.sigmas = defaultdict(list)
        self.weights = defaultdict(list)

        for c in range(self.classes):
            y_for_dualization = y_train[y_train != c]
            X_for_dualization = X_train[y_train != c]
            os.system('touch "in.txt"')
            with open('in.txt', 'w') as file:
                file.write(' '.join([str(elem) for elem in max_elems]) + '\n')
                for obj in X_for_dualization:
                    file.write(' '.join([str(elem) for elem in obj]) + '\n')

            #дуализация
            #print('dual start')
            os.system("g++ -std=c++17 -O3 final.cpp -o final && ./final")
            #print('dual end')
            coverages = pd.read_csv('out.csv', header=None).values
            sigmas = pd.read_csv('out_sigma.csv', header=None).values
            os.system("rm out.csv")
            os.system("rm out_sigma.csv")
            
            for i in range(len(max_elems)):
                sigmas[:, i][coverages[:, i] == 0] = max_elems[i]
            for i in range(len(max_elems), X_train.shape[1]):
                sigmas[:, i][coverages[:, i] == 0] = max_elems[i - len(max_elems)]

            #sigmas[coverages == 0] = self.max_sigma
            #print('here')
            #проверить на представительность
            for elcl, pos in zip(sigmas, coverages):
                good = 0
                for obj in X_train[y_train == c]:
                    if (elcl[np.where(pos == 1)[0]] >= obj[np.where(pos == 1)[0]]).all():
                        good += 1
                if good:
                    self.sigmas[c].append(elcl)
                    self.weights[c].append(good / np.sum([y_train == c]))



    def predict(self, X):
        #голосование 
        self.res = np.zeros((X.shape[0], self.classes))
        for c in range(self.classes):
            for i, obj in enumerate(X):

                for elcl, w in zip(self.sigmas[c], self.weights[c]):
                    if (obj <= elcl).all():
                        self.res[i][c] += w #вес - число прецедентов с таким эл кл

            if len(self.sigmas[c]):
                pass
                #self.res[:, c] /= len(self.sigmas[c])
        return self.res




In [153]:
def add_inverse(X, size):
    #a = np.arange(X.shape[1])
    #ind = np.random.choice(a, size=size, replace=False)
    ind = np.arange(size)
    newX = np.zeros((X.shape[0], X.shape[1] + len(ind)), dtype=np.int)
    newX[:, : X.shape[1]] = X
    for j, i in enumerate(ind):
        newX[:, X.shape[1] + j] = np.max(X[:, i]) - X[:, i]
    return newX

In [154]:
def enc(X, thr):
    newX = X.copy()
    inv = X.shape[1] - 1
    c = 0
    for i in range(X.shape[1]):
        encode = {k:j for j, k in enumerate(sorted(np.unique(X[:, i])))}
        if max(encode.values()) < thr:
            new_i = inv
            inv -= 1
        else:
            new_i = c
            c += 1
        for j in range(X.shape[0]):
            newX[j, new_i] = encode[X[j, i]]

    return newX, c

In [169]:
class Bagging_elcl(BaseEstimator):
    def __init__(self, max_elems, inv_feat, feature_subset=1):
        self.max_elems = max_elems
        self.feature_subset=feature_subset
        self.inv_feat = inv_feat
        
    def fit(self, X, y):        
        self.X_train = X
        self.y_train = y
        self.classes = np.max(y) + 1
    def predict(self, X_test, num_runs=10):
        res = np.zeros((X_test.shape[0], self.classes))
        for i in trange(num_runs):
            
            train_idx = np.random.randint(0, self.X_train.shape[0], self.X_train.shape[0])
            new_X_train = self.X_train[train_idx]
            new_y_train = self.y_train[train_idx]
            new_X_test = X_test
            tmp_max_elems = self.max_elems
            if self.feature_subset != 1:
                
                tmp = int((self.X_train.shape[1] - self.inv_feat) * self.feature_subset)
                feature_idx = np.sort(np.random.choice(self.X_train.shape[1] - self.inv_feat, tmp, 
                                                       replace=False))
                tmp_max_elems = self.max_elems[feature_idx]
                
                feature_idx = np.append(feature_idx, feature_idx + self.X_train.shape[1] - self.inv_feat)
                feature_idx = feature_idx[feature_idx < new_X_train.shape[1]]
                new_X_train = new_X_train[:, feature_idx]
                new_X_test = X_test[:, feature_idx]

                               
            
            elcl = Elcl()
            elcl.fit(new_X_train, new_y_train, self.classes, tmp_max_elems)
            
            res += elcl.predict(new_X_test)

        return np.argmax(res, axis=1)

In [156]:
class Bagging_trees(BaseEstimator):
    def __init__(self):
        return
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    def predict(self, X_test, num_runs=10):

        res = np.zeros((X_test.shape[0], np.max(self.y_train) + 1))
        for i in range(num_runs):

            train_idx = np.random.randint(0, self.X_train.shape[0], self.X_train.shape[0])
            new_X_train = self.X_train[train_idx]
            new_y_train = self.y_train[train_idx]

            tree = DecisionTreeClassifier(random_state=0, max_depth=5)
            tree.fit(new_X_train, new_y_train)
            res += tree.predict_proba(X_test)

        return np.argmax(res, axis=1)

In [157]:
def random_forest(X_train, y_train, X_test):
    forest = RandomForestClassifier()
    forest.fit(X_train, y_train)
    return forest.predict(X_test)

In [158]:
data_ph = pd.read_csv('datasets_81236_188723_ph-data.csv')
X = data_ph.drop(columns=['label']).values.astype(int)
y = data_ph[['label']].values[:, 0]

In [159]:
X1, c = enc(X, thr=3)
inv_feat = X1.shape[1]
Xd = add_inverse(X1, inv_feat)

cv = 3
feature_subset = 1
'm = ' + str(math.ceil(Xd.shape[0]*(cv-1)/cv)), 'k = ' + str(np.max(Xd)),\
'n = ' + str(int(Xd.shape[1] / 2 * feature_subset ) * 2)

('m = 436', 'k = 185', 'n = 6')

In [ ]:
model = Bagging_elcl(np.max(X1, axis=0), inv_feat=inv_feat, feature_subset=feature_subset)
ans = cross_val_predict(model, Xd, y, cv=cv)

model = Bagging_trees()
ans_tree = cross_val_predict(model, X, y)

model = RandomForestClassifier()
ans_forest = cross_val_predict(model, X, y)

np.mean(ans == y), np.mean(ans_tree == y), np.mean(ans_forest == y)

In [82]:
classes = np.arange(len(np.unique(y)))
tmp =  ohe(y, classes)

roc_auc_score(tmp, ohe(ans, classes), average='micro'), roc_auc_score(tmp, ohe(ans_tree, classes), average='micro'), \
roc_auc_score(tmp, ohe(ans_forest, classes), average='micro')

(0.5249945307372565, 0.8482279588711441, 0.853150295340188)

In [68]:
classes = np.arange(len(np.unique(y_test)))
tmp =  ohe(y_test, classes)

roc_auc_score(tmp, ohe(ans), average='macro'), roc_auc_score(tmp, ohe(ans_tree), average='macro'), \
roc_auc_score(tmp, ohe(ans_forest), average='macro')

TypeError: ohe() missing 1 required positional argument: 'classes'

In [161]:
data_heart = pd.read_csv('datasets_33180_43520_heart.csv')

X = data_heart.drop(columns=['target']).values.astype(int)
y = data_heart[['target']].values[:, 0]

X.shape, np.max(X)

((303, 13), 564)

In [178]:
X1, c = enc(X, thr=20)
inv_feat = X1.shape[1]
Xd = add_inverse(X1, inv_feat)
cv = 3
feature_subset = 0.5
'm = ' + str(math.ceil(Xd.shape[0]*(cv-1)/cv)), 'k = ' + str(np.max(Xd) + 1),\
'n = ' + str(int(Xd.shape[1] / 2 * feature_subset ) * 2)

('m = 1152', 'k = 4', 'n = 6')

In [170]:
model = Bagging_elcl(np.max(X1, axis=0), inv_feat=inv_feat, feature_subset=feature_subset)
ans = cross_val_predict(model, Xd, y, cv=cv)

model = Bagging_trees()
ans_tree = cross_val_predict(model, X, y)

model = RandomForestClassifier()
ans_forest = cross_val_predict(model, X, y)

np.mean(ans ==  y), np.mean(ans_tree == y), np.mean(ans_forest == y)

(0.8613861386138614, 0.6996699669966997, 0.8085808580858086)

In [171]:
classes = np.arange(len(np.unique(y)))
tmp =  ohe(y, classes)

roc_auc_score(tmp, ohe(ans, classes), average='macro'), roc_auc_score(tmp, ohe(ans_tree, classes), average='macro'), \
roc_auc_score(tmp, ohe(ans_forest, classes), average='macro')

(0.857905138339921, 0.6940052700922267, 0.805862977602108)

In [92]:
data_dermatology = pd.read_csv('dermatology.csv')

X = data_dermatology.drop(columns=['class']).values.astype(int)
y = data_dermatology[['class']].values[:, 0] - 1

In [93]:
X.shape

(366, 34)

In [180]:
X1, c = enc(X, thr=20)
inv_feat = X1.shape[1]
Xd = add_inverse(X1, inv_feat)
cv = 3
feature_subset = 0.5
'm = ' + str(math.ceil(Xd.shape[0]*(cv-1)/cv)), 'k = ' + str(np.max(Xd)+ 1),\
'n = ' + str(int(Xd.shape[1] / 2 * feature_subset ) * 2)

('m = 1152', 'k = 4', 'n = 6')

In [ ]:
model = Bagging_elcl(np.max(X1, axis=0), inv_feat=inv_feat, feature_subset=feature_subset)
ans = cross_val_predict(model, Xd, y, cv=cv)

model = Bagging_trees()
ans_tree = cross_val_predict(model, X, y, cv=cv)

model = RandomForestClassifier()
ans_forest = cross_val_predict(model, X, y, cv=cv)

np.mean(ans ==  y), np.mean(ans_tree == y), np.mean(ans_forest == y)


In [ ]:
classes = np.arange(len(np.unique(y)))
tmp =  ohe(y, classes)

roc_auc_score(tmp, ohe(ans, classes), average='macro'), roc_auc_score(tmp, ohe(ans_tree, classes), average='macro'), \
roc_auc_score(tmp, ohe(ans_forest, classes), average='macro')

In [172]:
data_car = pd.read_csv('car.data', header=None)

In [173]:
data_car.shape

(1728, 7)

In [174]:
for i in range(data_car.shape[1]):
    print(data_car[i].unique())

['vhigh' 'high' 'med' 'low']
['vhigh' 'high' 'med' 'low']
['2' '3' '4' '5more']
['2' '4' 'more']
['small' 'med' 'big']
['low' 'med' 'high']
['unacc' 'acc' 'vgood' 'good']


In [175]:
data_car = data_car.replace('low', 0)
data_car = data_car.replace('med', 1)
data_car = data_car.replace('high', 2)
data_car = data_car.replace('vhigh', 3)

data_car = data_car.replace('5more', 5)
data_car = data_car.replace('more', 5)

data_car = data_car.replace('small', 0)
data_car = data_car.replace('big', 2)

data_car = data_car.replace('unacc', 0)
data_car = data_car.replace('acc', 1)
data_car = data_car.replace('good', 2)
data_car = data_car.replace('vgood', 3)

In [176]:
X = data_car.values[:, :6].astype(int)
y = data_car[6].values

In [181]:
X1, c = enc(X, thr=3)
inv_feat = X1.shape[1]
Xd = add_inverse(X1, inv_feat)
cv = 5
feature_subset = 1
'm = ' + str(math.ceil(Xd.shape[0]*(cv-1)/cv)), 'k = ' + str(np.max(Xd) + 1),\
'n = ' + str(int(Xd.shape[1] / 2 * feature_subset ) * 2)

('m = 1383', 'k = 4', 'n = 12')

In [182]:
model = Bagging_elcl(np.max(X1, axis=0), inv_feat=inv_feat, feature_subset=feature_subset)
ans = cross_val_predict(model, Xd, y, cv=cv)

model = Bagging_trees()
ans_tree = cross_val_predict(model, X, y, cv=cv)

model = RandomForestClassifier()
ans_forest = cross_val_predict(model, X, y, cv=cv)

np.mean(ans ==  y), np.mean(ans_tree == y), np.mean(ans_forest == y)

(0.8078703703703703, 0.8298611111111112, 0.8211805555555556)

In [183]:
classes = np.arange(len(np.unique(y)))
tmp =  ohe(y, classes)

roc_auc_score(tmp, ohe(ans, classes), average='macro'), roc_auc_score(tmp, ohe(ans_tree, classes), average='macro'), \
roc_auc_score(tmp, ohe(ans_forest, classes), average='macro')

(0.776968534133446, 0.7424211079532403, 0.7895742378271532)